In [1]:
import numpy as np
import matplotlib.pyplot as plt

import obspy.taup

%matplotlib inline 

In [2]:
import earth_model

In [5]:
prem = earth_model.Prem()

In [26]:
# write out tvel model
# This is defined in read_tvel_file in https://github.com/obspy/obspy/blob/master/obspy/taup/velocity_model.py
# as:
#
#            comment line - generally info about the P velocity model
#            comment line - generally info about the S velocity model
#            depth pVel sVel Density
#            depth pVel sVel Density
# depths in km, velocities in km/s, density in g/cc (i.e. our density / 1000)
# we need to insert the discontiuities.

for i, bp in enumerate(prem.density_poly.breakpoints[-1::-1]):
    print(i, (prem.r_earth - bp), (prem.r_earth - prem.density_poly.breakpoints[i]))

0 0.0 6371.0
1 3.0 5149.5
2 15.0 2891.0
3 24.399999999999636 2741.0
4 80.0 771.0
5 220.0 670.0
6 400.0 600.0
7 600.0 400.0
8 670.0 220.0
9 771.0 80.0
10 2741.0 24.399999999999636
11 2891.0 15.0
12 5149.5 3.0
13 6371.0 0.0


In [36]:
nbps = len(prem.density_poly.breakpoints) -1
for i in range(nbps):
    print("layer", nbps - i)
    j = nbps - i
    k = j - 1
    print("Depth: ", prem.r_earth - prem.density_poly.breakpoints[j], "--", prem.r_earth - prem.density_poly.breakpoints[k])
    print("Radius: ", prem.density_poly.breakpoints[j], "--", prem.density_poly.breakpoints[k])

layer 13
Depth:  0.0 -- 3.0
Radius:  6371.0 -- 6368.0
layer 12
Depth:  3.0 -- 15.0
Radius:  6368.0 -- 6356.0
layer 11
Depth:  15.0 -- 24.399999999999636
Radius:  6356.0 -- 6346.6
layer 10
Depth:  24.399999999999636 -- 80.0
Radius:  6346.6 -- 6291.0
layer 9
Depth:  80.0 -- 220.0
Radius:  6291.0 -- 6151.0
layer 8
Depth:  220.0 -- 400.0
Radius:  6151.0 -- 5971.0
layer 7
Depth:  400.0 -- 600.0
Radius:  5971.0 -- 5771.0
layer 6
Depth:  600.0 -- 670.0
Radius:  5771.0 -- 5701.0
layer 5
Depth:  670.0 -- 771.0
Radius:  5701.0 -- 5600.0
layer 4
Depth:  771.0 -- 2741.0
Radius:  5600.0 -- 3630.0
layer 3
Depth:  2741.0 -- 2891.0
Radius:  3630.0 -- 3480.0
layer 2
Depth:  2891.0 -- 5149.5
Radius:  3480.0 -- 1221.5
layer 1
Depth:  5149.5 -- 6371.0
Radius:  1221.5 -- 0.0
